In [4]:
import numpy as np
import scde
import datasets as ds
import estimate_k as ek

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
x_train,lable = ds.get_mnist()
x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:]))) # make 28*28 img to 784 demission matrix

In [13]:
# estimate k by softmax
result = ek.estimate_mnist(20,x_train)
k = ek.get_k_from_result(result)

Epoch 1/5
235/235 [==============================] - 17s 72ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/5
235/235 [==============================] - 20s 83ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/5
235/235 [==============================] - 18s 77ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/5
235/235 [==============================] - 19s 79ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/5
235/235 [==============================] - 18s 76ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00


In [ ]:
sc_result = scde.clustering(x_train,10)

In [3]:
# make autoencoder
encoding_dim = 32
input_img = keras.Input(shape=(784,))
encoded = layers.Dense(encoding_dim, activation="relu")(input_img)
decoded = layers.Dense(784, activation="sigmoid")(encoded) 

# creater autoencoder
autoencoder = keras.Model(input_img, decoded)

# create encoder
encoder = keras.Model(input_img, encoded)

# creater decoder
encoded_input = keras.Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

NameError: name 'keras' is not defined

In [19]:
# complie and train autoencoder
autoencoder.compile(loss='mean_squared_error', optimizer='sgd')
autoencoder.fit(x_train, x_train, epochs=50, batch_size=256,shuffle=True)

Epoch 1/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 2/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 3/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 4/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 5/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 6/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 7/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 8/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 9/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 10/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 11/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 12/50
235/235 [==============================] - 1s 3ms/step - loss: 0.1101
Epoch 13/50
235/235 [====

In [60]:
# in first 100 points, all imgs in cluster 1
for i in range(100):
    if sc_result[i] == 9:
        print(lable[i])  

7
9
9
7
9
7
9
9
7
4
9
7
7
7
9
7
7
